Installing libraries

In [2]:
import stanza
import numpy as np
import pandas as pd

Importing bad words vocab and dataset

In [83]:
bad_words=[]
with open('./data/bad-words.csv') as f:
    for line in f.readlines():
        bad_words.append(line.split('\n')[0])

bad_words_set = set(bad_words)
dataset = pd.read_csv('comments2.csv')

Importing Stanza POS tagger

In [9]:
pos_tagger = stanza.Pipeline(lang='en', processors='tokenize,mwt,pos',use_gpu=True)

2023-03-15 13:31:58 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES
2023-03-15 13:31:58 WARNING: Can not find mwt: default from official model list. Ignoring it.
2023-03-15 13:31:59 INFO: Loading these models for language: en (English):
| Processor | Package  |
------------------------
| tokenize  | combined |
| pos       | combined |

2023-03-15 13:31:59 INFO: Using device: cuda
2023-03-15 13:31:59 INFO: Loading: tokenize
2023-03-15 13:31:59 INFO: Loading: pos
2023-03-15 13:31:59 INFO: Done loading processors!


POS Tags for our dataset

In [34]:
pos_tags_dataset =[]
for sentence in dataset.iloc[:,1].values.tolist():
    doc = pos_tagger(sentence)
    pos_tags_dataset.append([word.xpos for sent in doc.sentences for word in sent.words])

Writing the tags to a txt file (Simply)

In [37]:
with open('pos_tags_dataset.txt','w') as f:
    f.writelines([str(i) for i in pos_tags_dataset])

Labelling the dataset

In [91]:
labels= []
for i,sentence in enumerate(dataset['comments']):
    flag = False
    for word in sentence.split():
        if word.lower() in bad_words_set:
            flag = True
            continue
    labels.append(1) if flag else labels.append(0)

FUCK
crappy
assault
fucking
stupid
sick
Sucks
shit
Japanese
fire
explosion
fire
shit
fuck
sick
conspiracy
fuck
shit
bigger
conspiracy
WTF
American
fuck
Wtf
death
American
Fucking
Jesus
terrorist
died
shooting
sick
fuck
fuck
ass
fuck
fucked
shit
stupid
ass
shooting
American
god
angry
shit
nuke
sick
sick
sick
shits
fuck
fucking
fire
fuck
crap
shit
attack
sick
WTF
shit
shit
American
fucking
fuck
shooting
fuck
WTF
God
bombing
attack
shit
bi
fuck
shit
sick
Conspiracy
Republican
Jesus
pimp
Dick
fire
fuck
dumb
fuck
Wtf
death
fucking
hell
fucking
shit
shite
black
hole
fire
explosion
fire
wtf
Wtf
sick
dead
die
fuck
fuck
black
Fuck
killed
shit
fucking
sick
fire
Robber
idiot
attack
sick
fucking
shit
Christian
sick
shit
death
shooting
sick
crash
fire
failure
republican
FUCK
fire
corruption
bomb
killed
God
Shoot
shooting
fight
FUCK
American
die
death
death
damn
FUCK
kill
conspiracy
attack
Fucking
damn
failed
fire
dumb
fuck
sick
harder
FUCK
Fuckin
suck
fucking
Shits
American
ass
republican
republica

Percentage of bad sentences

In [ ]:
np.count_nonzero(labels)/len(labels)

0.2533157894736842